We'll download the dataset first:

In [1]:
import tensorflow as tf

(X_train, Y_train), (X_validation, Y_validation) = \
    tf.keras.datasets.cifar10.load_data()

Y_train = tf.keras.utils.to_categorical(Y_train, 10)
Y_validation = tf.keras.utils.to_categorical(Y_validation, 10)

170498071/170498071 [==============================] - 14s 0us/step


Next, we'll instantiate a DataGenerator, which will normalize the train and test datasets, and will provide data augmentation during training:

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    horizontal_flip=True,
    vertical_flip=True)

# Apply z-normalization on the training set
data_generator.fit(X_train)

# Standardize the validation set
X_validation = data_generator.standardize(X_validation.astype('float32'))

Then, we'll define the CNN model:

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, BatchNormalization, Activation, Flatten

model = Sequential(layers=[
    Conv2D(32, (3, 3),
           padding='same',
           input_shape=X_train.shape[1:]),
    BatchNormalization(),
    Activation('gelu'),
    Conv2D(32, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('gelu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),

    Conv2D(64, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('gelu'),
    Conv2D(64, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('gelu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),

    Conv2D(128, (3, 3)),
    BatchNormalization(),
    Activation('gelu'),
    Conv2D(128, (3, 3)),
    BatchNormalization(),
    Activation('gelu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.5),

    Flatten(),
    Dense(10, activation='softmax')
])

We'll define the optimization parameters:

In [4]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 32, 32, 32)        0

Finally, we'll run the training and evaluation:

In [5]:
batch_size = 50

model.fit(
    x=data_generator.flow(x=X_train,
                          y=Y_train,
                          batch_size=batch_size),
    steps_per_epoch=len(X_train) // batch_size,
    epochs=100,
    verbose=1,
    validation_data=(X_validation, Y_validation),
    workers=4)

Epoch 1/100
1000/1000 [==============================] - 27s 15ms/step - loss: 1.6768 - accuracy: 0.4029 - val_loss: 1.3441 - val_accuracy: 0.5032
Epoch 2/100
1000/1000 [==============================] - 15s 15ms/step - loss: 1.2025 - accuracy: 0.5667 - val_loss: 1.1317 - val_accuracy: 0.5964
Epoch 3/100
1000/1000 [==============================] - 15s 15ms/step - loss: 1.0374 - accuracy: 0.6295 - val_loss: 0.8895 - val_accuracy: 0.6794
Epoch 4/100
1000/1000 [==============================] - 15s 15ms/step - loss: 0.9352 - accuracy: 0.6662 - val_loss: 0.9180 - val_accuracy: 0.6747
Epoch 5/100
1000/1000 [==============================] - 15s 15ms/step - loss: 0.8696 - accuracy: 0.6903 - val_loss: 0.7886 - val_accuracy: 0.7207
Epoch 6/100
1000/1000 [==============================] - 15s 15ms/step - loss: 0.8214 - accuracy: 0.7087 - val_loss: 0.8486 - val_accuracy: 0.6986
Epoch 7/100
1000/1000 [==============================] - 14s 14ms/step - loss: 0.7867 - accuracy: 0.7223 - val_loss: 0